<a href="https://colab.research.google.com/github/Nirbhai/Learning-MLOps-1/blob/main/toolkit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python

In [1]:
!pip install rich

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 232 kB 5.0 MB/s 
     |████████████████████████████████| 51 kB 6.5 MB/s 


In [2]:
from rich import print as rprint

## Decorators

Functions allow us to modularize code and reuse them. However, we'll often want to add some functionality before or after the main function executes and we may want to do this for many different functions. Instead of adding more code to the original function, we can use `decorators`!

`decorators`: augment a function with pre/post-processing. Decorators wrap around the main function and allow us to operate on the inputs and or outputs.

Suppose we have a function called `addTaxToPayment` which changes the status of a payment when invoked.

In [3]:
def addTaxToPayment(payment):
    """Adds 10% tax to the Payment"""
    tax = 0.1 * payment
    payment += tax
    return payment

In [4]:
rprint (addTaxToPayment(100))

110.0

Now let's say we want to print the payment total before and after the `addTaxToPayment` function executes. Here's how we would do it by changing the original code of the function:

In [5]:
def addTaxToPayment(payment):
    """Adds 1-% tax to the Payment"""
    rprint (payment)
    tax = 0.1 * payment
    payment += tax
    rprint (payment)
    return payment

In [6]:
addTaxToPayment(100)

100

110.0

110.0

We were able to achieve what we want but we now increased the size of our `addTaxToPayment` function and if we want to do the same printing for any other function, we have to add the same code to all of those as well ... not very efficient. To solve this, let's create a decorator called `printValue` which prints the value of payment before and after the main function `addTaxToPayment` executes.

**Creating a Decorator function**

The decorator function accepts a function `f` which is the function we wish to wrap around, in our case, it's `addTaxToPayment()`. The output of the decorator is its wrapper function which receives the arguments and keyword arguments passed to function `f`.

Inside the wrapper function, we: 
1. extract the input parameters passed to function `f`. 
2. make any changes we want to the function inputs. 
3. execute function f 
4. make any changes to the function outputs 
5. wrapper function returns some value(s), which is what the decorator returns as well since it returns wrapper.

In [7]:
# Decorator
def printValue(f):
    def wrapper(*args, **kwargs):
        """Wrapper function for @addTaxToPayment."""
        payment = kwargs.get("payment") # use .pop() if altering payment & add the argument back in function call arguments in line 7
        rprint (f"value before function call {payment}") # executes before function f
        payment = f(*args, **kwargs)
        rprint (f"value after function call {payment}") # executes after function f
        return payment
    return wrapper

We can use this decorator by simply adding it to the top of our main function preceded by the `@` symbol.

In [8]:
@printValue
def addTaxToPayment(payment):
    """Adds 10% tax to the Payment"""
    tax = 0.1 * payment
    payment += tax
    return payment

In [9]:
addTaxToPayment(payment = 100)

value before function call 100

value after function call 110.0

110.0

Suppose we wanted to debug and see what function actually executed with `addTaxToPayment()`.

In [10]:
addTaxToPayment.__name__, addTaxToPayment.__doc__

('wrapper', 'Wrapper function for @addTaxToPayment.')

The function name and docstring are not what we're looking for but it appears this way because the wrapper function is what was executed. In order to fix this, Python offers `functools.wraps` which carries the main function's metadata.

In [11]:
from functools import wraps

# Decorator
def printValue(f):
    @wraps(f)
    def wrap(*args, **kwargs):
        """Wrapper function for @addTaxToPayment."""
        payment = kwargs.get("payment") # use .pop() if altering payment & add the argument back in function call arguments in line 7
        rprint (f"value before function call {payment}") # executes before function f
        payment = f(*args, **kwargs)
        rprint (f"value after function call {payment}") # executes after function f
        return payment
    return wrap

@printValue
def addTaxToPayment(payment):
    """Adds 10% tax to the Payment"""
    tax = 0.1 * payment
    payment += tax
    return payment

addTaxToPayment.__name__, addTaxToPayment.__doc__

('addTaxToPayment', 'Adds 10% tax to the Payment')

Awesome! We were able to decorate our main function `addTaxToPayment()` to achieve the customization we wanted without actually altering the function. We can reuse our decorator for other functions that may need the same customization!

This was a dummy example to show how decorators work. We can use decorators in MLOps: a simple scenario would be using decorators to create uniform JSON responses from each API endpoint without including the bulky code in each endpoint.



## Callbacks

Decorators allow for customized operations before and after the main function's execution but what about in between? Suppose we want to conditionally/situationally do some operations. Instead of writing a whole bunch of if-statements and make our functions bulky, we can use callbacks!

**callbacks**: conditional/situational processing within the function.

Here callbacks will be classes that have functions with key names that will execute at various periods during the main function's execution. The function names are up to us but we need to invoke the same callback functions within our main function.

In [12]:
# Callback
class payment_tracker(object):
    def __init__(self, payment):
        self.history = []
    def at_start(self, payment):
        self.history.append(payment)
    def at_end(self, payment):
        self.history.append(payment)

We can pass in as many callbacks as we want and because they have appropriately named functions they will be invoked at the appropriate times.

In [13]:
def addTaxToPayment(payment, callbacks = []):
    """Adds 10% tax to the Payment"""
    for callback in callbacks:
        callback.at_start(payment)
    tax = 0.1 * payment
    payment += tax
    for callback in callbacks:
        callback.at_end(payment)
    return payment

In [14]:
payment = 100
tracker = payment_tracker(payment=payment)
addTaxToPayment(payment=payment, callbacks=[tracker])
rprint (tracker.history)

[100, 110.0]

##  Difference compared to a decorator?

With callbacks, it's easier to keep track of objects since it's all defined in a separate callback class. It's also now possible to interact with our function, not just before or after but throughout the entire process! Imagine a function with:

- multiple processes where we want to execute operations in between them
- execute operations repeatedly when loops are involved in functions

## Classes

Classes are object constructors and are a fundamental component of object oriented programming in Python. They are composed of a set of functions that define the class and it's operations.

### Magic methods

Classes can be customized with magic methods like __init__ and __str__, to enable powerful operations. These are also known as `dunder` methods (ex. dunder init), which stands for `d`ouble `under`scores due to the leading and trailing underscores.

#### `__init__` function

The `__init__` function is used when an instance of the class is initialized. 

In [15]:
# Creating the class
class Payment(object):
    """Class object for a payment."""
    
    def __init__(self, mode, name):
        """Initialize a payment."""
        self.mode = mode
        self.name = name

In [16]:
# Creating an instance of a class
my_payment = Payment(mode="CreditCard",
             name="amazon_order")
rprint (my_payment)
rprint (my_payment.name)


<__main__.Payment object at 0x7f4049ccf710>

amazon_order

#### `__str__` function

The `print (my_payment)` command printed something not so relevant to us. Let's fix that with the `__str__` function.

In [17]:
# Creating the class
class Payment(object):
    """Class object for a payment."""
    
    def __init__(self, mode, name):
        """Initialize a payment."""
        self.mode = mode
        self.name = name
    
    def __str__(self):
        """Output this when print an instance of a Payment"""
        return f"payment mode for {self.name} is {self.mode}"

Let's try print the instance of `Payment` class again.

In [18]:
# Creating an instance of a class
my_payment = Payment(mode="CreditCard",
             name="amazon_order")
rprint (my_payment)
rprint (my_payment.name)


payment mode for amazon_order is CreditCard

amazon_order

#### Few other magic/dunder methods

- `__len__`
- `__iter__`
- `__getitem__`

### Object functions

Besides these magic functions, classes can also have object functions.

In [19]:
# Creating the class
class Payment(object):
    """Class object for a payment."""
    
    def __init__(self, mode, name):
        """Initialize a payment."""
        self.mode = mode
        self.name = name
    
    def __str__(self):
        """Output this when print an instance of a Payment"""
        return f"payment mode for {self.name} is {self.mode}"
    
    def changeName(self, new_name):
        """Change the name of your Payment"""
        self.name = new_name

In [20]:
# Creating an instance of a class
my_payment = Payment(mode="CreditCard",
             name="amazon_order")
rprint (my_payment)
rprint (my_payment.name)

payment mode for amazon_order is CreditCard

amazon_order

In [21]:
# Using a class's object function
my_payment.changeName(new_name="uber_order")
rprint (my_payment)
rprint (my_payment.name)

payment mode for uber_order is CreditCard

uber_order

### Inheritance

We can also build classes on top of one another using inheritance, which allows us to inherit all the properties and methods from another class (the parent).

In [22]:
class CreditCardPayment(Payment):
    def __init__(self, name, status):
        super().__init__(mode="CreditCard", name=name)
        self.status = status

    def __str__(self):
        return f"{self.name} is {self.status}"

In [23]:
orderPayment = CreditCardPayment(status="under_process", name="amazon_order")
rprint (orderPayment)

amazon_order is under_process

In [24]:
orderPayment.changeName("amazon_order_1")
rprint (orderPayment)

amazon_order_1 is under_process

- Notice how we inherited the initialized variables from the parent `Payment` class like `payment_mode` and `name`. We also inherited functions such as `changeName()`.
- As you can see, both our parent class (`Payment`) and the child class (`CreditCardPayment`) have different `__str__` functions defined but share the same function name. The child class inherits everything from the parent classes but when there is conflict between function names, the child class' functions take precedence and overwrite the parent class' functions.

### Decorator Methods

There are two important decorator methods to know about when it comes to classes: `@classmethod` and `@staticmethod`

In [25]:
# Creating the class
class Payment(object):
    """Class object for a payment."""
    
    def __init__(self, mode, name):
        """Initialize a payment."""
        self.mode = mode
        self.name = name
    
    def __str__(self):
        """Output this when print an instance of a Payment"""
        return f"payment mode for {self.name} is {self.mode}"
    
    @classmethod
    def from_dict(cls, d):
        return cls(name=d["name"], mode=d["mode"])
    
    @staticmethod
    def is_cashOnDelivery(mode):
        return mode == "COD"

A `@classmethod` allows us to create class instances by passing in the uninstantiated class itself (`cls`). This is a great way to create (or load) classes from objects (ie. dictionaries).

In [26]:
# create instance
d = { "name" : "amazon_oder",
      "mode" : "CreditCard"}
orderPayment = Payment.from_dict(d)
rprint (orderPayment) 

payment mode for amazon_oder is CreditCard

A `@staticmethod` can be called from an uninstantiated class object so we can do things like this:

In [27]:
rprint (Payment.is_cashOnDelivery(mode = "COD"))
rprint (Payment.is_cashOnDelivery(mode = "CreditCard"))

True

False